<a href="https://colab.research.google.com/github/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/1-essentials-of-nlp/2_text_vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vectorizing text

To understand how to process text, it is important to understand the general
workflow for NLP.

<img src='https://github.com/rahiakela/img-repo/blob/master/advanced-nlp-with-tensorflow-2/text-processing-workflow.png?raw=1' width='800'/>

The first two steps of the process in the preceding diagram involve collecting labeled data. A supervised model or even a semi-supervised model needs data to operate.

The next step is usually normalizing and featurizing the data. Models have a hard time processing text data as is. There is a lot of hidden structure in a given text that needs to be processed and exposed. These two steps focus on that. 

There are a couple of challenges in using the text content of messages. **The first is that text can be of arbitrary lengths.**

Comparing this to image data, we know that each image has a fixed width and height. Even if the corpus of images has a mixture of sizes, images
can be resized to a common size with minimal loss of information by using a variety of compression mechanisms.

In NLP, this is a bigger problem compared to computer vision. A common approach to handle this is to truncate the text.

**The second issue is that of the representation of words with a numerical quantity or feature.**

In computer vision, the smallest unit is a pixel. Each pixel has a set of
numerical values indicating color or intensity. 

In a text, the smallest unit could be a word. Aggregating the Unicode values of the characters does not convey or embody the meaning of the word.

**A core problem then is to construct a numerical representation of words. Vectorization is the process of converting a word to a vector of numbers that embodies the information contained in the word. Depending on the vectorization technique, this vector may have additional properties that may allow comparison with other words.**

These are the followings text vectorization approach:

- **Count-based vectorization**: The simplest approach for vectorizing is to use counts of words.
- **TF-IDF based text vectorization**: This is more sophisticated, with its origins in information retrieval.
- **Word2Vec based text vectorization**: it generate embeddings or word vectors.
- **BERT based text vectorization**: The newest method in this area.


## Setup

In [ ]:
%tensorflow_version 2.x     # magic command instructing to use TensorFlow version 2+
import tensorflow as tf

import os
import io
import re

import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

from keras.utils import np_utils

tf.__version__

In [2]:
# Basic 1-layer neural network model for evaluation
def make_model(input_dims=3, num_units=12):
  model = tf.keras.Sequential()

  # Adds a densely-connected layer with 12 units to the model:
  model.add(tf.keras.layers.Dense(num_units, 
                                  input_dim=input_dims, 
                                  activation='relu'))

  # Add a sigmoid layer with a binary output unit:
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

## Data collection

**The first step of any Machine Learning (ML) project is to obtain a dataset.**

We will be using the SMS Spam Collection dataset made available by University of California, Irvine.

In [ ]:
# Download the zip file
path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
                  origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
                  extract=True)

# Unzip the file into a folder
!unzip $path_to_zip -d data

In [ ]:
# optional step - helps if colab gets disconnected
# from google.colab import drive
# drive.mount('/content/drive')

Reading the data file is trivial.

In [ ]:
# Let's see if we read the data correctly
# lines = io.open('/content/drive/My Drive/colab-data/SMSSpamCollection').read().strip().split('\n')
lines = io.open('/content/data/SMSSpamCollection').read().strip().split('\n')
lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
lines[2]

"spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

### Pre-process Data

The next step is to split each line into two columns – one with the text of the message and the other as the label. While we are separating these labels, we will also convert the labels to numeric values. Since we are interested in predicting spam messages, we can assign a value of 1 to the spam
messages. A value of 0 will be assigned to legitimate messages.

In [ ]:
spam_dataset = []
spam_count = 0
ham_count = 0
for line in lines:
  label, text = line.split('\t')
  if label.lower().strip() == 'spam':
    spam_dataset.append((1, text.strip()))
    spam_count += 1
  else:
    spam_dataset.append(((0, text.strip())))
    ham_count += 1

spam_dataset[:5]

[(0,
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'),
 (0, 'Ok lar... Joking wif u oni...'),
 (1,
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"),
 (0, 'U dun say so early hor... U c already then say...'),
 (0, "Nah I don't think he goes to usf, he lives around here though")]

In [ ]:
print("Spam: ", spam_count, ", Ham: ", ham_count)

Spam:  747 , Ham:  4827


Now the dataset is ready for further processing in the pipeline.

## Count-based vectorization


The idea behind count-based vectorization is really simple. Each unique word
appearing in the corpus is assigned a column in the vocabulary. Each document,
which would correspond to individual messages in the spam example, is assigned
a row. The counts of the words appearing in that document are entered in
the relevant cell corresponding to the document and the word. 

**With $n$ unique documents containing $m$ unique words, this results in a matrix of $n$ rows by $m$ columns.**

Consider a corpus like so:

In [4]:
corpus = [
  "I like fruits. Fruits like bananas",
  "I love bananas but eat an apple",
  "An apple a day keeps the doctor away"
]

There are three documents in this corpus of text. The scikit-learn (sklearn)
library provides methods for undertaking count-based vectorization.

The `CountVectorizer` class provides a built-in tokenizer that separates the tokens of two or more characters in length. This class takes a variety of options including a custom tokenizer, a stop word list, the option to convert characters to lowercase prior to tokenization, and a binary mode that converts every positive count to 1.

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

vectorizer.get_feature_names()

['an',
 'apple',
 'away',
 'bananas',
 'but',
 'day',
 'doctor',
 'eat',
 'fruits',
 'keeps',
 'like',
 'love',
 'the']

The full matrix can be seen as follows:

In [6]:
X.toarray()

array([[0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]])

This process has now converted a sentence such as "I like fruits. Fruits like bananas" into a vector `(0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0)`.

**This is an example of context free vectorization. Context-free refers to the fact that the order of the words in the document did not make any difference in the generation of the vector. This is merely counting the instances of the words in a document.**

Consequently, words with multiple meanings may be grouped into one, for example, bank. This may refer to a place near the river or a place to keep money. 

**However, it does provide a method to compare documents and derive similarity. The cosine similarity or distance can be computed between two documents, to see which documents are similar to which other documents**:

In [7]:
cosine_similarity(X.toarray())

array([[1.        , 0.13608276, 0.        ],
       [0.13608276, 1.        , 0.3086067 ],
       [0.        , 0.3086067 , 1.        ]])

This shows that the first sentence and the second sentence have a `0.136` similarity score (on a scale of 0 to 1). The first and third sentence have nothing in common. The second and third sentence have a similarity score of `0.308` – the highest in this set.

**Another use case of this technique is to check the similarity of the documents
with given keywords.**

Let's say that the query is apple and bananas. This first step is
to compute the vector of this query, and then compute the cosine similarity scores against the documents in the corpus:

In [8]:
query = vectorizer.transform(["apple and bananas"])

cosine_similarity(X, query)

array([[0.23570226],
       [0.57735027],
       [0.26726124]])

This shows that this query matches the second sentence in the corpus the best. The third sentence would rank second, and the first sentence would rank lowest.

In a few lines, a basic search engine has been implemented, along with logic to serve queries!

## TF-IDF Vectorization

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X.toarray())

pd.DataFrame(tfidf.toarray(), 
             columns=vectorizer.get_feature_names())

In [ ]:
tfidf = TfidfVectorizer(binary=True)
X = tfidf.fit_transform(train['Message']).astype('float32')
X_test = tfidf.transform(test['Message']).astype('float32')

In [ ]:
X.shape

In [ ]:
from keras.utils import np_utils

_, cols = X.shape
model2 = make_model(cols)  # to match tf-idf dimensions
lb = LabelEncoder()
y = lb.fit_transform(y_train)
dummy_y_train = np_utils.to_categorical(y)
model2.fit(X.toarray(), y_train, epochs=10, batch_size=10)

In [ ]:
model2.evaluate(X_test.toarray(), y_test)

In [ ]:
train.loc[train.Spam == 1].describe() 

# Word Vectors

In [ ]:
# memory limit may be exceeded. Try deleting some objects before running this next section
# or copy this section to a different notebook.
!pip install gensim

In [ ]:
api.info()

In [ ]:
model_w2v = api.load("word2vec-google-news-300")

In [ ]:
model_w2v.most_similar("cookies",topn=10)

In [ ]:
model_w2v.doesnt_match(["USA","Canada","India","Tokyo"])

In [ ]:
king = model_w2v['king']
man = model_w2v['man']
woman = model_w2v['woman']

queen = king - man + woman  
model_w2v.similar_by_vector(queen)